# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [190]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import textdistance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [191]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]


def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=10, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_with_freq = [(word, dist, P(text, N)) for word, dist in closest]
    # сначала сортируем по расстоянию, потом по частоте встречаемости
    return sorted(closest_with_freq, key=lambda x: (x[1], x[2]), reverse=True)


N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [192]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('wiki_data.txt', encoding='utf8').read()

In [193]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)


In [194]:
get_closest_hybrid_match('солнечный', X, vec)

[('солнечный', 1.0, 4.8493980636159555e-06),
 ('солнечные', 0.8888888888888888, 4.8493980636159555e-06),
 ('солнечным', 0.8888888888888888, 4.8493980636159555e-06),
 ('солнечных', 0.8888888888888888, 4.8493980636159555e-06),
 ('солнечной', 0.8888888888888888, 4.8493980636159555e-06),
 ('солнечными', 0.8, 4.8493980636159555e-06),
 ('конечный', 0.7777777777777778, 4.8493980636159555e-06),
 ('солнечно', 0.7777777777777778, 4.8493980636159555e-06),
 ('солнечное', 0.7777777777777778, 4.8493980636159555e-06),
 ('соленый', 0.7777777777777778, 4.8493980636159555e-06)]

In [195]:
get_closest_hybrid_match('солнце', X, vec)

[('солнце', 1.0, 2.4440966240624417e-05),
 ('солнцем', 0.8571428571428572, 2.4440966240624417e-05),
 ('солнцев', 0.8571428571428572, 2.4440966240624417e-05),
 ('солнцу', 0.8333333333333334, 2.4440966240624417e-05),
 ('солнца', 0.8333333333333334, 2.4440966240624417e-05),
 ('солнцеву', 0.75, 2.4440966240624417e-05),
 ('солнцева', 0.75, 2.4440966240624417e-05),
 ('солонцы', 0.7142857142857143, 2.4440966240624417e-05),
 ('соляное', 0.7142857142857143, 2.4440966240624417e-05),
 ('солнцевой', 0.6666666666666667, 2.4440966240624417e-05)]

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [198]:
correct_word = list(vocab.keys())
deleted_char = {}
for word in correct_word:
    for i in range(1, len(word) + 1):
        tmp = word[:i-1] + word[i:]
        if tmp in deleted_char.keys():
            deleted_char[tmp].append(word)
        else:  
            deleted_char[tmp] = [word]
#     deleted_char.update(temp_dict)


In [207]:
list(deleted_char.items())[:15]

[('овостройка', ['новостройка']),
 ('нвостройка', ['новостройка']),
 ('ноостройка', ['новостройка']),
 ('новстройка', ['новостройка']),
 ('новотройка', ['новостройка']),
 ('новосройка', ['новостройка']),
 ('новостойка', ['новостройка']),
 ('новострйка', ['новостройка']),
 ('новострока', ['новостройка']),
 ('новостройа', ['новостройка']),
 ('новостройк', ['новостройка', 'новостройки']),
 ('ижегородская', ['нижегородская']),
 ('нжегородская', ['нижегородская']),
 ('ниегородская', ['нижегородская']),
 ('нижгородская', ['нижегородская'])]

In [201]:
def fix_error(text, deleted_char):
    similar = set()
    for i in range(1, len(text) + 1):
        if text[:i - 1] + text[i:] in deleted_char.keys():
            for word in deleted_char[text[:i - 1] + text[i:]]:
                similar.add(word)
    similar_freq = [(word, P(word, N)) for word in similar]
    if len(similar_freq) == 0:
        return " "
    return sorted(similar_freq, key=lambda x: x[1], reverse=True)


In [202]:
fix_error('солнце', deleted_char)

[('солнца', 3.220000314240995e-05),
 ('солнце', 2.4440966240624417e-05),
 ('солнцу', 3.1036147607142114e-06),
 ('олонце', 3.879518450892764e-07)]

In [203]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], fix_error(pair[1], deleted_char)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [204]:
print(correct / total)
print(mistaken_fixed / total_mistaken)
print(correct_broken / total_correct) 

0.8185092546273136
0.20031055900621117
0.09004249454461927


## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [177]:
def get_metrics(func, *args, **kwargs):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], func(pair[1], X, vec, *args, **kwargs)[0][0])
                cashed[pair[1]] = pred
            else:
                pred = pair[1]


            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)
    return correct / total, mistaken_fixed / total_mistaken, correct_broken / total_correct

### Посчитаем качество стандартного алгоритма, на дефолтных параметрах:

In [178]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)


In [179]:
res = get_metrics(get_closest_hybrid_match)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.473
correct_broken/total_correct: 0.09


### 1) Слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить до 30-ти:

In [180]:
res = get_metrics(get_closest_hybrid_match, topn=30)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.474
correct_broken/total_correct: 0.09


Изменения незначительные (mistaken_fixed/total_mistaken: 0.473 -> 0.474), попробуем увеличить topn еще сильнее

### 2) Слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить до 50-ти:

In [181]:
res = get_metrics(get_closest_hybrid_match, topn=50)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.475
correct_broken/total_correct: 0.09


Изменения опять незначительные, но чуть лучше Эксперимента 1; mistaken_fixed/total_mistaken: 0.473 -> 0.475 по сравнению со стандартными значениями параметров)

### 3) Может метод подсчёта расстояний влияет на правильность интерпретации разницы между словами, рассмотрим textdistance.hamming:

In [182]:
res = get_metrics(get_closest_hybrid_match, metric=textdistance.hamming)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.826
mistaken_fixed/total_mistaken: 0.258
correct_broken/total_correct: 0.09


Результат изменения подсчета функции расстояния ощутимо хуже результатов стандартной функции Левенштейна

### 4) Может метод подсчёта расстояний влияет на правильность интерпретации разницы между словами, рассмотрим textdistance.cosine:

In [183]:
res = get_metrics(get_closest_hybrid_match, metric=textdistance.cosine)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.841
mistaken_fixed/total_mistaken: 0.375
correct_broken/total_correct: 0.09


Функция косинусного расстояния показала результаты лучше, чем функция Хэмминга, но все равно хуже, чем стандартная функция

### 5) Может метод подсчёта расстояний влияет на правильность интерпретации разницы между словами, рассмотрим textdistance.strcmp95:

In [184]:
res = get_metrics(get_closest_hybrid_match, metric=textdistance.strcmp95)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.849
mistaken_fixed/total_mistaken: 0.439
correct_broken/total_correct: 0.09


Функция подсчета расстояния между строками показала хороший результат, но все равно хуже, чем стандратный результат

### 6) Попробуем увеличить max_features при векторизации, таким образом сохранив больше информации о словах (1000 -> 2000):

In [185]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=2000)
X = vec.fit_transform(vocab)

res = get_metrics(get_closest_hybrid_match)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.477
correct_broken/total_correct: 0.09


Результаты ощутимо лучше, попробуем увеличить max_features еще сильнее

### 7) Попробуем увеличить max_features при векторизации, таким образом сохранив больше информации о словах (1000 -> 4000):

In [186]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=4000)
X = vec.fit_transform(vocab)

res = get_metrics(get_closest_hybrid_match)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.855
mistaken_fixed/total_mistaken: 0.482
correct_broken/total_correct: 0.09


Наверное, увеличение max_features еще сильнее улучшит результат, но это считается слишком долго (

### 8) Попробуем увеличить ngram_range при векторизации, это поможет рассмотреть больше различных комбинаций букв ( (1, 3) -> (1, 4) ):

In [187]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=1000)
X = vec.fit_transform(vocab)

res = get_metrics(get_closest_hybrid_match)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.853
mistaken_fixed/total_mistaken: 0.471
correct_broken/total_correct: 0.09


Увеличение количества рассматриваемых токенов путем увеличения их длины не улучшило результат

### 9) С помощью комбинации увеличения ngram_range и max_features при векторизации можно добится что чуть более редкие n-граммы всё равно будут рассматриватся в дальнейшем при векторизации ( (1, 3) -> (1, 4); 1000 -> 2000 ):

In [188]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=2000)
X = vec.fit_transform(vocab)

res = get_metrics(get_closest_hybrid_match)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.475
correct_broken/total_correct: 0.09


Комбинация увеличения длины ngram и max_features улучшила результат, но это связано с тем, что само по себе увеличение max_features показывает еще бОльший результат, поэтому увеличение длины ngram не улучшает качество 

### 10) С помощью комбинации увеличения topn при отборе и max_features при векторизации можно добится что больше слов будут рассматриваться после перемножения увеличенных по размерности векторов ( 20 -> 50; 1000 -> 2000 ):

In [189]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=2000)
X = vec.fit_transform(vocab)

res = get_metrics(get_closest_hybrid_match, topn=50)

print(f"correct/total: {round(res[0], 3)}")
print(f"mistaken_fixed/total_mistaken: {round(res[1], 3)}")
print(f"correct_broken/total_correct: {round(res[2], 3)}")

0
100
200
300
400
500
600
700
800
900
correct/total: 0.854
mistaken_fixed/total_mistaken: 0.474
correct_broken/total_correct: 0.09


Комбинация увеличения topn и max_features не дает прироста в качестве

### Лучший результат – способ увеличения max_features (рассматривается больше различных ngram, увеличивается размерность векторов). Остальные эксперименты не привели к желаемому улучшению.